### transformer based language model

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
	text = f.read()

len(text)

1115394

In [10]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [15]:
# get vocab
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [58]:
# tokenization
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda ixs: ''.join([itos[i] for i in ixs])

In [19]:
# encode data
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:10])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [21]:
# split up data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [36]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
	data = train_data if split == 'train' else val_data
	ix = torch.randint(len(data) - block_size, (batch_size,))
	x = torch.stack([data[i:i+block_size] for i in ix])
	y = torch.stack([data[i+1:i+block_size+1] for i in ix])
	return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

# for b in range(batch_size):
# 	for t in range(block_size):
# 		context = xb[b, :t+1]
# 		target = yb[b, t]
# 		print(f"when input is {context}, the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [63]:
class BigramLanguageModel(nn.Module):
	
	def __init__(self, vocab_size):
		super().__init__()
		self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  
	def forward(self, idx, targets=None):
		logits = self.token_embedding_table(idx)
  
		if targets is None:
			loss = None
		else:
			B, T, C = logits.shape
			logits = logits.view(B*T, C)
			targets = targets.view(B*T)
			loss = F.cross_entropy(logits, targets)
   
		return logits, loss

	def generate(self, idx, max_new_tokens):
		
		for _ in range(max_new_tokens):
			logits, loss = self(idx)
			logits = logits[:, -1, :]
			probs = F.softmax(logits, dim=-1)
			idx_next = torch.multinomial(probs, num_samples=1)
			idx = torch.cat((idx,idx_next), dim=1)
		return idx

		




m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8945, grad_fn=<NllLossBackward0>)



ZfLdaugGL-IrbNHTAXHBF VQpW,ci;:ZCuj,cx?$$cpo-WLC?Fc-vHp'cCh$:-LYypIgtKUxQQVDyIUt:VN;w!yeGEa!,LUObzKf


In [65]:
# create optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [74]:
# training
batch_size = 32

for steps in range(10000):
	xb, yb = get_batch('train')
 
	logits, loss = m(xb, yb)
	optimizer.zero_grad(set_to_none=True)
	loss.backward()
	optimizer.step()
print(loss.item())

2.4307925701141357


In [76]:
# inference
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Bessamara thence,
I fur, told ph, thetorouthe,

DWhou moo pofarisp theas the thesa-
ABleendasary yoriklourbe t fissit, hee,

Tous ar's, kertu andosir:
To anssirecrwhatine wsothe m thinkiththat ch n ollos ngishe he hende, k he er.
An Louslloatowifod ft! se:
Goumel y be thesorple grirfo fo, ds he, for
